In [13]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [14]:
df = pd.read_csv(Path('./data/20210227.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,2/23/2021 18:54:04,Miami Blues (1990),Miami Connection (1987),Lady in Cement (1968),Step Up Revolution (2012),"DAMN, THAT'S SOME SLEAZY SHIT!",WATCH A DOCUMENTARY YOU COWARDS,DE PALMA-RAMA,ANIMALS THAT /TALK/,[Default to Nicolas Cage]
1,2/23/2021 19:05:09,Step Up Revolution (2012),Miami Connection (1987),Miami Blues (1990),Lady in Cement (1968),WATCH A DOCUMENTARY YOU COWARDS,DE PALMA-RAMA,"DAMN, THAT'S SOME SLEAZY SHIT!",ANIMALS THAT /TALK/,[Default to Nicolas Cage]
2,2/25/2021 9:20:31,Miami Blues (1990),Step Up Revolution (2012),Miami Connection (1987),Lady in Cement (1968),WATCH A DOCUMENTARY YOU COWARDS,[Default to Nicolas Cage],ANIMALS THAT /TALK/,DE PALMA-RAMA,"DAMN, THAT'S SOME SLEAZY SHIT!"
3,2/23/2021 21:43:42,Step Up Revolution (2012),Miami Blues (1990),Miami Connection (1987),Lady in Cement (1968),WATCH A DOCUMENTARY YOU COWARDS,DE PALMA-RAMA,ANIMALS THAT /TALK/,"DAMN, THAT'S SOME SLEAZY SHIT!",[Default to Nicolas Cage]
4,2/23/2021 21:43:56,Lady in Cement (1968),Miami Blues (1990),Miami Connection (1987),Step Up Revolution (2012),WATCH A DOCUMENTARY YOU COWARDS,"DAMN, THAT'S SOME SLEAZY SHIT!",DE PALMA-RAMA,ANIMALS THAT /TALK/,[Default to Nicolas Cage]
5,2/23/2021 21:53:44,Step Up Revolution (2012),Miami Blues (1990),Lady in Cement (1968),Miami Connection (1987),ANIMALS THAT /TALK/,"DAMN, THAT'S SOME SLEAZY SHIT!",WATCH A DOCUMENTARY YOU COWARDS,DE PALMA-RAMA,[Default to Nicolas Cage]
6,2/23/2021 22:10:04,Miami Blues (1990),Lady in Cement (1968),NaN,NaN,DE PALMA-RAMA,WATCH A DOCUMENTARY YOU COWARDS,"DAMN, THAT'S SOME SLEAZY SHIT!",[Default to Nicolas Cage],ANIMALS THAT /TALK/
7,2/24/2021 4:50:51,Step Up Revolution (2012),Miami Connection (1987),Miami Blues (1990),Lady in Cement (1968),WATCH A DOCUMENTARY YOU COWARDS,DE PALMA-RAMA,[Default to Nicolas Cage],"DAMN, THAT'S SOME SLEAZY SHIT!",ANIMALS THAT /TALK/
8,2/24/2021 7:26:15,Step Up Revolution (2012),Miami Blues (1990),Miami Connection (1987),Lady in Cement (1968),ANIMALS THAT /TALK/,"DAMN, THAT'S SOME SLEAZY SHIT!",DE PALMA-RAMA,WATCH A DOCUMENTARY YOU COWARDS,[Default to Nicolas Cage]
9,2/24/2021 8:42:06,Miami Connection (1987),NaN,NaN,NaN,"DAMN, THAT'S SOME SLEAZY SHIT!",ANIMALS THAT /TALK/,NaN,NaN,NaN


In [15]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [16]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Miami Blues (1990)', 'Miami Connection (1987)',
       'Lady in Cement (1968)', 'Step Up Revolution (2012)'], dtype=object)

In [17]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [18]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [19]:
submission_dict

{'Miami Blues (1990)': <Candidate('Miami Blues (1990)')>,
 'Miami Connection (1987)': <Candidate('Miami Connection (1987)')>,
 'Lady in Cement (1968)': <Candidate('Lady in Cement (1968)')>,
 'Step Up Revolution (2012)': <Candidate('Step Up Revolution (2012)')>}

In [20]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [21]:
ballots

[<Ballot(Miami Blues (1990), Miami Connection (1987), Lady in Cement (1968), Step Up Revolution (2012))>,
 <Ballot(Step Up Revolution (2012), Miami Connection (1987), Miami Blues (1990), Lady in Cement (1968))>,
 <Ballot(Miami Blues (1990), Step Up Revolution (2012), Miami Connection (1987), Lady in Cement (1968))>,
 <Ballot(Step Up Revolution (2012), Miami Blues (1990), Miami Connection (1987), Lady in Cement (1968))>,
 <Ballot(Lady in Cement (1968), Miami Blues (1990), Miami Connection (1987), Step Up Revolution (2012))>,
 <Ballot(Step Up Revolution (2012), Miami Blues (1990), Lady in Cement (1968), Miami Connection (1987))>,
 <Ballot(Miami Blues (1990), Lady in Cement (1968))>,
 <Ballot(Step Up Revolution (2012), Miami Connection (1987), Miami Blues (1990), Lady in Cement (1968))>,
 <Ballot(Step Up Revolution (2012), Miami Blues (1990), Miami Connection (1987), Lady in Cement (1968))>,
 <Ballot(Miami Connection (1987))>,
 <Ballot(Miami Connection (1987), Miami Blues (1990), Lady in 

In [22]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [23]:
print(election_result)

ROUND 1
Candidate                    Votes  Status
-------------------------  -------  --------
Step Up Revolution (2012)        5  Hopeful
Miami Blues (1990)               3  Hopeful
Miami Connection (1987)          2  Rejected
Lady in Cement (1968)            1  Rejected

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
Miami Blues (1990)               5  Elected
Step Up Revolution (2012)        5  Rejected
Miami Connection (1987)          0  Rejected
Lady in Cement (1968)            0  Rejected
Blank votes                      1  Rejected



In [24]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                          Votes  Status
-------------------------------  -------  --------
WATCH A DOCUMENTARY YOU COWARDS        5  Hopeful
DAMN, THAT'S SOME SLEAZY SHIT!         3  Hopeful
ANIMALS THAT /TALK/                    2  Rejected
DE PALMA-RAMA                          1  Rejected
[Default to Nicolas Cage]              0  Rejected

FINAL RESULT
Candidate                          Votes  Status
-------------------------------  -------  --------
WATCH A DOCUMENTARY YOU COWARDS        6  Elected
DAMN, THAT'S SOME SLEAZY SHIT!         5  Rejected
ANIMALS THAT /TALK/                    0  Rejected
DE PALMA-RAMA                          0  Rejected
[Default to Nicolas Cage]              0  Rejected

